# OpenStreetMap Data Case Study
## by Sergio Marfull
__________________________________________

## Map Area

Madrid, Spain

* https://www.openstreetmap.org/relation/5326784

I was born in Madrid and I decided to investigate the dataset of my city and, who knows, I may help the OpenStreetMap community someday to get an improved database!

## Setup and preparing the dataset

In [1]:
## import all the necessary packages to run the code

import sqlite3 as sqlite
import xml.etree.cElementTree as ET
from pprint import pprint
import re
from collections import defaultdict
from time import time
import mapparser
import tags
import users
import audit 
import schema
import data
import cerberus
import csv
import codecs

Open the dataset and store it in order to manipulate it in python

In [2]:
## store the dataset from the .osm file into a python variable
osm_file = open("OSM Madrid - XML/madrid_spain.osm","rb")

Create a smaller sample that can be inspected faster before investigating the entire dataset

In [ ]:
## create a sample file, 1000 times smaller than the original file in order to speed up the 
## investigation of the dataset and the iterations of the code
sample_file = "OSM Madrid - XML/sample.osm"
t0=time()
k = 1000 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(sample_file, 'w') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(osm_file)):
        if i % k == 0:
            output.write(ET.tostring(element).decode('UTF-8'))

    output.write('</osm>')
print(time()-t0)

### Number of tags

In [10]:
osm_file.seek(0)
mapparser.count_tags(osm_file)

{'bounds': 1,
 'member': 168733,
 'nd': 6097035,
 'node': 4734085,
 'osm': 1,
 'relation': 16293,
 'tag': 2670626,
 'way': 615917}

### Types of tags

It is important to see what kind of tags we have in the dataset. For example, we don't have any problematic character and we can see that almost 2 million tags are lowercase, and almost 1 million have a colon in the middle.

In [12]:
osm_file.seek(0)
tags.process_map(osm_file)

{'lower': 1836786, 'lower_colon': 780841, 'other': 52999, 'problemchars': 0}

## Problems found in the Map
________________________________________________-

I have encountered so many challenges for this project. First of all, it is the first project of a kind in my life and I have gone through all existing problems of types, formats and coding. Regarding dataset problems, I encountered many syntactical problems which a fixed with the same function (audit.py), namely:

* **Abbreviated street names** (Avenida, Avda., AV.,...)
* **Misspelling** (Carretetera, punctuation,...)
* **Incongruent versions of the same street names, mixing lower and uppercases** (Calle, calle, CALLE)

I have also had one special problem: the language. In Spain, we call the street type at the beginning, and all the references that I found explained the English-speaking case (even when I asked directly in some forums). So I had to practice with the regular expression by myself until I got with the formula:


In [4]:
street_type_re = re.compile(r'^\w+\.?', re.IGNORECASE)

Since that moment, I could accelerate the project path again and correct the words replacing them for their mapping in audit.py 

In [ ]:
def audit(filename):
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(filename, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    return street_types

> All helper functions used in the code can be found included within the .zip file of the project

## Audit the data

We used the test function (within audit.py) in order to replace the old words with the mapping list and improve the dataset before exporting it. We can see in the output the different first words of the tags, as I explained earlier, due to the Spanish nomenclature. Here, we can see the similar words and all the tags affected for those errors, so we can decide what changes are more relevant or have more weight in order to prioritize the audit task.

In [3]:
osm_file.seek(0)
audit.test(osm_file)

{'11': {'11 Posterior'},
 'A': {'A-5', 'A-4 KM 23.3'},
 'AUTOP.': {'AUTOP. VALENCIA KM 7.1'},
 'AUTOPISTA': {'AUTOPISTA AUTOPISTA AP-41 KM 31.780',
               'AUTOPISTA AUTOPISTA AP41 KM 31.760'},
 'AUTOVIA': {'AUTOVIA A-1 KM 29.1',
             'AUTOVIA A-3 KM 62.5',
             'AUTOVIA A-3 KM 71.9',
             'AUTOVIA A-42 KM 57.700',
             'AUTOVIA E-901/A-3 KM 35.2',
             'AUTOVIA NACIONAL I KM 26.200'},
 'AV': {'AV del Deporte', 'AV DE MOSTOLES, Nº 6'},
 'AV.': {'AV. CASTILLA'},
 'AVDA': {'AVDA ARCAS DE AGUA SN',
          'AVDA DE MADRID',
          'AVDA DEL VALLE 36 (CRTA M 604 KM 24.100)'},
 'AVDA.': {'AVDA. ALTO DEL LEÓN, Nº 3',
           'AVDA. DEL ANTIGÜO FERROCARRIL, Nº 1',
           'AVDA. GENERALISIMO,12',
           'AVDA. SAN PABLO'},
 'AVENIDA': {'AVENIDA ANDALUCIA',
             'AVENIDA AURELIO ALVAREZ, S/N',
             'AVENIDA AVDA DEL SOL ESQ.AVDA DE FRANCIA, SN',
             'AVENIDA AVDA. DE LA CABRERA S/N JUNTO AL HOSTAL EL CANCHO

CR C-9, 31 (C-9 KM 31.4) => Carretera C-9, 31 (C-9 KM 31.4)
CR A-42, 47,3 => Carretera A-42, 47,3
CR A-5, 23.5 => Carretera A-5, 23.5
CR A-3 MG IZDO, KM 11.2 KM 11.200 => Carretera A-3 MG IZDO, KM 11.2 KM 11.200
CR A-5 => Carretera A-5
CR M-506 MOSTOLES-FUENLABRADA KM. 4 => Carretera M-506 MOSTOLES-FUENLABRADA KM. 4
CR M-214, 4,6 => Carretera M-214, 4,6
CR N-4 P.K. 6,70 I => Carretera N-4 P.K. 6,70 I
CARRETERA M-509 KM 4.4 => Carretera M-509 KM 4.4
CARRETERA M 506 KM 4.900 => Carretera M 506 KM 4.900
CARRETERA M-404 KM 39.8 => Carretera M-404 KM 39.8
CARRETERA MADRID-TOLEDO KM 57.700 => Carretera MADRID-TOLEDO KM 57.700
CARRETERA VALDEMAQUEDA KM. 0 => Carretera VALDEMAQUEDA KM. 0
CARRETERA ANDALUCIA KM 28.230 => Carretera ANDALUCIA KM 28.230
CARRETERA TOLEDO -CUESTA LA REINA KM 10.200 => Carretera TOLEDO -CUESTA LA REINA KM 10.200
CARRETERA M-623 KM 1.5 => Carretera M-623 KM 1.5
CARRETERA M-305 KM. 5 => Carretera M-305 KM. 5
CARRETERA M-233 KM 6.800 => Carretera M-233 KM 6.800
CARRETER

## Export data to .CSV files and create the database

Once we have audited our OpenStreetMap dataset, we are going to export it to 5 .csv data files (nodes, nodes_tags, ways, ways_tags and ways_nodes).

In [7]:
osm_file.seek(0)
data.process_map(osm_file, validate = True)

In order to investigate those files, we are creating a database with 5 tables using sqlite and the following schema:

In [ ]:
#importing cleaned .csv files to SQL schema
CREATE TABLE nodes (
    id INTEGER,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);

CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);

CREATE TABLE ways (
    id INTEGER,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);

CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);

CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);

## File size

madrid_spain.osm..........1.05 GB

osm_madrid.db.............955 MB

nodes.csv.................499 MB

nodes_tags.csv............53 MB

ways.csv..................48 MB

ways_tags.csv.............75 MB

ways_nodes.csv............185 MB

## Number of unique users

In [ ]:
sqlite> SELECT COUNT(DISTINCT(e.uid))          
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;

3290

## Number of nodes

In [ ]:
sqlite> SELECT COUNT(*) FROM nodes;

4734085

## Number of ways

In [ ]:
sqlite> SELECT COUNT(*) FROM ways;

615917

## Top 10 contributing users

In [ ]:
sqlite> SELECT i.user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) i
GROUP BY i.user
ORDER BY num DESC
LIMIT 10;

carlosz22 **514913**

rafaerti **410891**

cirdancarpintero **364599**

JavierSp **230958**

Serfuen **192328**

robertogeb **171717**

gpesquero **159991**

Pelanas **158102**

sergionaranja **140543**

mor **133633**

If we don't group by i.user, then we get the total number of contributions, which is 5350002. According to this, the best collaborator accounted for 9.6% of the total contributions. The Top-5 accounted for the 32% and Top-10 for 46%.

## Top 10 Amenities in Madrid

In [ ]:
sqlite> SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='amenity'
GROUP BY value
ORDER BY num DESC
LIMIT 10;

restaurant **4494**

drinking_water **2391**

pharmacy **2322**

bench  **2033**

bar **1931**

bank **1930**

school **1561**

recycling **1377**

bicycle_parking **1293**

cafe **1186**

## Top 10 cuisines in Madrid

In [ ]:
sqlite> SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value= 'restaurant') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='cuisine'
GROUP BY nodes_tags.value
ORDER BY num DESC;

regional **740**

Spanish **204**

Chinese **142**

Italian **134**

burger **83**

japanese **79**

pizza **72**

Asian **68**

Mexican **65**

Indian **35**

## Additional improvements and benefits

There is lot of room for improvement in this database. If we look at the audit function output, we can further appreciate other problems that could be studied in next steps. For example, we see that the street type has been omitted in some cases, starting directly with a name, when it was most probably preceeded by "Calle" (street in English).

We should confront this output with another database in order to correct the wrong inputs and, then, our analysis would be more complete and accurate!

## Conclusions

It has been a very satisfying project that revealed the room for standardization and improvement of a collaborative dataset as OpenStreetMap is. With a relatively small piece of code, we have been able to normalize hundreds of street names and find some more issues that could be fixed with a workteam in very little time with a bit of effort. 

For me, it has been very inspiring about what we can do in the future by learning those skills and makes me think of ways to collaborate with my modest knowledge so far :)